In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Path to image folders, 
data_path = fr'../Datasets/Test Set'
# os.listdir(data_path)

def remove_white_background(pixels):
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0

    return pixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]



def blueify(pixels):
    return [b for r, g, b in pixels]


def process_files(src):
    data = dict()
    X_red = []
    X_green = []
    X_blue = []
    y = np.empty(shape=[0, 1])
    print("Starting...")
    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            print("Converting image..")
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            X_red.append(redify(pixels.copy()))
            X_green.append(greenify(pixels.copy()))
            X_blue.append(blueify(pixels.copy()))
            y = np.append(y, subdir)
    print("Finished")
    return np.asarray(X_red), np.asarray(X_green), np.asarray(X_blue), y

X_red, X_green, X_blue, y = process_files(data_path)
print(X_red)
print(X_green)
print(X_blue)
print(y)


Starting...
Finished
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[255 255 255 ... 254 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
['Tomato 3' 'Tomato 3' 'Tomato 3']


In [10]:
# Path to "train" folder, which has all our images
all_data_path = fr'../Datasets/train'
X_red_all, X_green_all, X_blue_all, y_all = process_files(all_data_path)

# Split set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_red_all, y_all, test_size=0.20, random_state=40)

print(f'Training set shape: {X_train.shape}'); print(f'Test set shape: {X_test.shape}')

# Fit test dataset and run predictions
from sklearn import svm
from sklearn.neural_network import MLPClassifier

mlpc_red =  MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_red.fit(X_train, y_train)

predict_train = mlpc_red.predict(X_train)
predict_test = mlpc_red.predict(X_test)


Starting...
Finished
Training set shape: (2308, 576)
Test set shape: (577, 576)


In [11]:
# Performance metrics
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[298   0   0   0   0   0]
 [  0 387   0   0   0   0]
 [  0   0 369   0   0   0]
 [  0   0   0 302   0   0]
 [  0   0   0   0 362   0]
 [  0   0   0   0   0 590]]
                   precision    recall  f1-score   support

         Tomato 3       1.00      1.00      1.00       298
         Tomato 4       1.00      1.00      1.00       387
Tomato Cherry Red       1.00      1.00      1.00       369
    Tomato Maroon       1.00      1.00      1.00       302
    Tomato Yellow       1.00      1.00      1.00       362
           Walnut       1.00      1.00      1.00       590

         accuracy                           1.00      2308
        macro avg       1.00      1.00      1.00      2308
     weighted avg       1.00      1.00      1.00      2308

